# RL Swarm Worker Node (Google Colab)

This notebook runs a **worker node** that:
- Participates in training
- Coordinates with the coordinator node via Google Drive
- Shares rollouts via Google Drive (no P2P networking)
- No blockchain, Docker, or peer identity required

**Before running:**
1. Ensure coordinator node is running (using `colab_coordinator.ipynb`)
2. Mount your Google Drive (same account as coordinator)
3. Set **same EXPERIMENT_NAME** as coordinator
4. Set **unique NODE_ID** for this worker
5. Run all cells in order

**To run multiple workers:** Use this notebook in separate Colab sessions with different NODE_IDs

## 1. Configuration

In [ ]:
# Experiment Configuration
EXPERIMENT_NAME = 'qwen_0.6b_seed42'  # MUST MATCH COORDINATOR
NODE_ROLE = 'worker'  # DO NOT CHANGE
NODE_ID = 'worker_1'  # MUST BE UNIQUE (worker_1, worker_2, worker_3, etc.)

# Model Configuration
MODEL_NAME = 'Gensyn/Qwen2.5-0.5B-Instruct'  # Should match coordinator
SEED = 42  # Should match coordinator

# Training Configuration
MAX_ROUNDS = 1000
NUM_GENERATIONS = 2
NUM_TRANSPLANT_TREES = 2

# Rollout Sharing Configuration (should match coordinator)
ROLLOUT_PUBLISH_FREQUENCY = 'stage'  # 'generation', 'stage', or 'round'
ROLLOUT_CLEANUP_ENABLED = False      # Set to True to enable cleanup
ROLLOUT_KEEP_LAST_N_ROUNDS = 10      # Only used if cleanup enabled
ROLLOUT_ARCHIVE_OLD = False          # Archive instead of delete

# Optional: HuggingFace Token (for pushing trained models)
HUGGINGFACE_TOKEN = None  # Set to your token or keep None

# Optional: Wandb Configuration
WANDB_API_KEY = None  # Set to your Wandb API key or keep None
WANDB_PROJECT = 'rl-swarm-colab'

print(f"âœ“ Experiment: {EXPERIMENT_NAME}")
print(f"âœ“ Node Role: {NODE_ROLE}")
print(f"âœ“ Node ID: {NODE_ID}")
print(f"âœ“ Model: {MODEL_NAME}")
print()
print("âڑ  Make sure EXPERIMENT_NAME matches the coordinator!")

## 2. Mount Google Drive

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set base path (must be same as coordinator)
GDRIVE_BASE_PATH = '/content/drive/MyDrive/rl-swarm'

# Check if experiment exists
experiment_path = os.path.join(GDRIVE_BASE_PATH, 'experiments', EXPERIMENT_NAME)
if not os.path.exists(experiment_path):
    print(f"â‌Œ Experiment '{EXPERIMENT_NAME}' not found!")
    print(f"   Expected at: {experiment_path}")
    print()
    print("Make sure:")
    print("  1. Coordinator is running")
    print("  2. EXPERIMENT_NAME matches the coordinator")
    print("  3. You're using the same Google Drive account")
    raise FileNotFoundError(f"Experiment not found: {EXPERIMENT_NAME}")
else:
    print(f"âœ“ Found experiment: {EXPERIMENT_NAME}")
    print(f"  Path: {experiment_path}")

## 3. System Setup & Dependencies

In [ ]:
# Check GPU availability
import torch

if torch.cuda.is_available():
    print(f"âœ“ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("âڑ  No GPU detected - training will be slow")
    print("  Consider: Runtime > Change runtime type > GPU")

In [ ]:
# Clone repository
import os

# Change to safe directory first
%cd /content

# Remove existing directory if it exists
if os.path.exists('/content/rl-swarm'):
    print("Removing existing repository...")
    !rm -rf /content/rl-swarm

# Clone fresh copy
print("Cloning repository...")
!git clone https://github.com/Elrashid/rl-swarm.git /content/rl-swarm

# Change to repo directory
%cd /content/rl-swarm

# Verify clone worked
if not os.path.exists('requirements.txt'):
    print("❌ Clone failed! requirements.txt not found")
    raise FileNotFoundError("Repository clone failed")

print("✓ Repository cloned successfully")

# Install dependencies
print("Installing dependencies (this may take 3-5 minutes)...")
!pip install -q -r requirements.txt
!pip install -q gensyn-genrl==0.1.9

print("✓ Dependencies installed")

In [ ]:
if WANDB_API_KEY:
    import wandb
    wandb.login(key=WANDB_API_KEY)
    print("âœ“ Wandb configured")
else:
    print("â„¹ Wandb disabled (WANDB_API_KEY not set)")

## 6. Set Environment Variables

In [ ]:
import os

# Set environment variables
os.environ['GDRIVE_PATH'] = GDRIVE_BASE_PATH
os.environ['EXPERIMENT_NAME'] = EXPERIMENT_NAME
os.environ['NODE_ROLE'] = NODE_ROLE
os.environ['NODE_ID'] = NODE_ID
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['SEED'] = str(SEED)

if HUGGINGFACE_TOKEN:
    os.environ['HUGGINGFACE_ACCESS_TOKEN'] = HUGGINGFACE_TOKEN

if WANDB_API_KEY:
    os.environ['WANDB_API_KEY'] = WANDB_API_KEY
    os.environ['WANDB_PROJECT'] = WANDB_PROJECT

print("âœ“ Environment variables set")

## 7. Check Peer Discovery

Verify that we can discover the coordinator and other workers.

In [ ]:
import os
import uuid

# Set environment variables
os.environ['GDRIVE_PATH'] = GDRIVE_BASE_PATH
os.environ['EXPERIMENT_NAME'] = EXPERIMENT_NAME
os.environ['NODE_ROLE'] = NODE_ROLE
os.environ['NODE_ID'] = NODE_ID or f"worker_{uuid.uuid4().hex[:8]}"
os.environ['MODEL_NAME'] = MODEL_NAME
os.environ['SEED'] = str(SEED)

# Rollout configuration
os.environ['ROLLOUT_PUBLISH_FREQUENCY'] = ROLLOUT_PUBLISH_FREQUENCY
os.environ['ROLLOUT_CLEANUP_ENABLED'] = str(ROLLOUT_CLEANUP_ENABLED)
os.environ['ROLLOUT_KEEP_LAST_N_ROUNDS'] = str(ROLLOUT_KEEP_LAST_N_ROUNDS)
os.environ['ROLLOUT_ARCHIVE_OLD'] = str(ROLLOUT_ARCHIVE_OLD)

if HUGGINGFACE_TOKEN:
    os.environ['HUGGINGFACE_ACCESS_TOKEN'] = HUGGINGFACE_TOKEN

if WANDB_API_KEY:
    os.environ['WANDB_API_KEY'] = WANDB_API_KEY
    os.environ['WANDB_PROJECT'] = WANDB_PROJECT

print("âœ“ Environment variables set")
print(f"  Node ID: {os.environ['NODE_ID']}")

In [ ]:
import sys
import subprocess

print("="*60)
print(f"Starting Worker Node: {NODE_ID}")
print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Model: {MODEL_NAME}")
print("="*60)
print()

# Run training
try:
    subprocess.run([
        sys.executable, '-m', 'rgym_exp.runner.swarm_launcher',
        '--config-name', 'colab-gdrive'
    ])
    
except KeyboardInterrupt:
    print("" + "="*60)
    print("Training interrupted by user")
    print("="*60)
except Exception as e:
    print(f"❌ Error: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
from rgym_exp.utils.experiment_manager import get_experiment_status, get_experiment_metrics
import pandas as pd

# Get current status
status = get_experiment_status(GDRIVE_BASE_PATH, EXPERIMENT_NAME)

print(f"Experiment: {EXPERIMENT_NAME}")
print(f"Current Round: {status.get('current_round', 0)}")
print(f"Current Stage: {status.get('current_stage', 0)}")
print(f"Active Peers: {status.get('active_peers', 0)}")

if 'peer_ids' in status:
    print(f"Peer IDs: {', '.join(status['peer_ids'])}")

if 'total_metric_entries' in status:
    print(f"Total Metric Entries: {status['total_metric_entries']}")

if 'latest_checkpoint' in status:
    print(f"Latest Checkpoint: Round {status['latest_checkpoint']}")

if 'error' in status:
    print(f"⚠️  Error: {status['error']}")

print()

# Load and display recent metrics for this node
try:
    df = get_experiment_metrics(GDRIVE_BASE_PATH, EXPERIMENT_NAME)
    if not df.empty:
        print(f"Recent metrics for {NODE_ID} (last 10 rounds):")
        recent = df[df['node_id'] == NODE_ID].tail(10)
        if not recent.empty:
            print(recent[['round', 'stage', 'my_reward']].to_string(index=False))
        else:
            print(f"No metrics for {NODE_ID} yet")
    else:
        print("No metrics available yet - training may not have started")
except Exception as e:
    print(f"Could not load metrics: {e}")

## 10. Resume Training (If Disconnected)

If your Colab session disconnects:
1. Re-run all cells above (keep same EXPERIMENT_NAME and NODE_ID)
2. The system will automatically resume from the last checkpoint
3. Training continues from the last saved round
4. Peer discovery will reconnect to coordinator automatically

## Troubleshooting

### Worker can't find experiment
- Verify EXPERIMENT_NAME matches coordinator exactly
- Check that coordinator has finished cell 7 (Initialize Experiment)
- Ensure using same Google Drive account

### No peers discovered
- Coordinator must be running and past cell 8
- Wait 1-2 minutes for peer discovery to propagate

### Out of memory errors
- Use smaller model (e.g., Qwen2.5-0.5B instead of 1.5B)
- Reduce NUM_GENERATIONS to 1
- Enable GPU: Runtime > Change runtime type > GPU

### Training too slow
- Check GPU is enabled and available
- Reduce model size
- Check coordinator round duration isn't too short